# Exercise 3: Durable Agent — Solution

Complete implementation combining **OpenAI Agents SDK** with **Temporal** for durability.

This solution demonstrates how Temporal automatically retries failed operations, making your AI agents resilient to network failures and other transient errors.

## Architecture

This solution follows a **4-component pattern** that mirrors production Temporal applications:

1. **`activities.py`** - Defines the weather activity
2. **`workflow.py`** - Defines the workflow orchestration
3. **`worker.py`** - Runs the worker that executes workflows/activities
4. **`starter.py`** - Executes the workflow

Each component is implemented in a separate Jupyter cell below for clarity.

## Prerequisites

1. **Temporal dev server must be running:**
   ```bash
   temporal server start-dev
   ```

2. **OpenAI API key configured:**
   ```bash
   export OPENAI_API_KEY=sk-your-key-here
   ```

3. **Internet connection:** Required for National Weather Service API

## Setup

Install dependencies:

In [ ]:
%pip install --quiet temporalio openai-agents httpx nest-asyncio pytz

# Import all required modules
import asyncio
import httpx
from datetime import timedelta, datetime
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib import openai_agents
from temporalio.contrib.openai_agents import OpenAIAgentsPlugin, ModelActivityParameters
from temporalio.worker.workflow_sandbox import SandboxedWorkflowRunner, SandboxRestrictions
from agents import Agent, Runner

print("✅ All imports successful")

## Component 1: `activities.py`

Define the weather activity that fetches alerts from the National Weather Service API.

**Key points:**
- Returns a `dict` (not formatted string) so the LLM can interpret the data
- Activity name explicitly set with `@activity.defn(name="get_weather")`
- Simple error handling - just `raise_for_status()` and let Temporal retry
- No console logging or attempt tracking needed

In [ ]:
@activity.defn(name="get_weather")
async def get_weather(state: str) -> dict:
    """Fetch active NWS alerts for a 2-letter US state code (e.g., 'CA')."""
    headers = {"User-Agent": "Temporal-Agents-Workshop/1.0 (educational@example.com)"}
    async with httpx.AsyncClient(timeout=10) as client:
        r = await client.get(
            f"https://api.weather.gov/alerts/active/area/{state}",
            headers=headers
        )
        r.raise_for_status()
        data = r.json()

    alerts = []
    for f in (data.get("features") or [])[:5]:
        p = f.get("properties", {})
        alerts.append({
            "event": p.get("event"),
            "headline": p.get("headline"),
            "severity": p.get("severity"),
            "area": p.get("areaDesc"),
        })

    return {"state": state.upper(), "count": len(alerts), "alerts": alerts}

print("✅ Activity 'get_weather' defined")

In [ ]:
## Component 2: `workflow.py`

Define the workflow that orchestrates the agent execution.

**Key points:**
- Agent is created INSIDE the workflow (not globally)
- Uses `activity_as_tool()` to convert Temporal activity into an Agent tool
- Single parameter: `user_query` (no trace_id needed)
- `Runner()` is instantiated (not just `Runner`)
- Uses `getattr()` for safe access to `final_output`

In [ ]:
TASK_QUEUE = "weather-agents"

@workflow.defn
class WeatherAgentWorkflow:
    @workflow.run
    async def run(self, user_query: str) -> str:
        # Create the Agent inside the workflow
        agent = Agent(
            name="Weather Assistant",
            instructions=(
                "You are a helpful assistant that explains current weather alerts for U.S. states. "
                "When the user asks about alerts, call the get_weather tool, then summarize results."
            ),
            tools=[
                # Convert Temporal activity to Agent tool
                openai_agents.workflow.activity_as_tool(
                    get_weather,
                    start_to_close_timeout=timedelta(seconds=10),
                )
            ],
        )
        
        # Run the agent - tool calls execute as Temporal activities
        result = await Runner().run(agent, user_query)
        
        # Safely extract final output
        return getattr(result, "final_output", str(result))

print(f"✅ Workflow 'WeatherAgentWorkflow' defined (task queue: {TASK_QUEUE})")

## Component 3: `worker.py`

Run the worker that polls for and executes workflow/activity tasks.

**Key points:**
- Connects to Temporal with `OpenAIAgentsPlugin` (required)
- Configures `ModelActivityParameters` for AI model timeout settings
- Uses `SandboxedWorkflowRunner` with passthrough for `httpx`
- Runs in background as an async task in Jupyter

In [ ]:
async def run_worker():
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",
        plugins=[
            OpenAIAgentsPlugin(
                # Configure timeout for AI model inference activities
                model_params=ModelActivityParameters(
                    start_to_close_timeout=timedelta(seconds=30)
                )
            )
        ],
    )

    # Create worker that polls the task queue
    worker = Worker(
        client,
        task_queue=TASK_QUEUE,
        workflows=[WeatherAgentWorkflow],
        activities=[get_weather],
        # Configure sandbox to allow httpx module (needed for NWS API calls)
        workflow_runner=SandboxedWorkflowRunner(
            restrictions=SandboxRestrictions.default.with_passthrough_modules(
                "httpx",  # Allow httpx to bypass sandbox restrictions
            )
        )
    )
    
    print(f"✅ Worker started on task queue: {TASK_QUEUE}")
    print("   Listening for workflow and activity tasks...")
    await worker.run()

# Start worker in background (Jupyter-specific pattern)
import nest_asyncio
nest_asyncio.apply()
worker_task = asyncio.create_task(run_worker())
print("🔄 Worker running in background")

## Component 4: `starter.py`

Execute the workflow and display results.

**Key points:**
- Uses `start_workflow` (not `execute_workflow`) for better control
- Workflow ID uses EST timestamp pattern
- Single query parameter (no trace_id)
- Waits for result with `handle.result()`

### 🎯 Demonstrating Durability (Optional)

To see Temporal's automatic retry in action:

1. **Run the cell below** to start the workflow
2. **While it's executing, disconnect your network** (WiFi off or disable network adapter)
3. **Watch the logs** - You'll see the activity fail and Temporal retry
4. **Reconnect your network**
5. **Observe** Temporal automatically complete the workflow on the next retry!

In [ ]:
async def run_solution():
    """Execute the weather agent workflow."""
    query = "What weather alerts are active in CA?"
    
    # Generate workflow ID with EST timestamp
    est = pytz.timezone('US/Eastern')
    now = datetime.now(est)
    workflow_id = f"weather-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"
    
    # Connect to Temporal with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",
        plugins=[OpenAIAgentsPlugin()]
    )
    
    print(f"🚀 Starting workflow: {workflow_id}")
    print(f"📝 Query: {query}\n")
    
    # Start the workflow (non-blocking)
    handle = await client.start_workflow(
        WeatherAgentWorkflow.run,
        query,
        id=workflow_id,
        task_queue=TASK_QUEUE
    )
    
    print(f"✅ Workflow started: {handle.id}")
    print(f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n")
    print("⏳ Waiting for agent response...\n")
    
    # Wait for workflow to complete and get result
    result = await handle.result()
    
    print("=" * 60)
    print("🤖 Agent Response:")
    print("=" * 60)
    print(result)
    print("=" * 60)

# Run the solution (Jupyter-specific async handling)
try:
    loop = asyncio.get_running_loop()
    await run_solution()
except RuntimeError:
    asyncio.run(run_solution())